## Notebook to apply regression model (Step 3)

In [1]:
import os
import glob
import datetime
from osgeo import gdal
import numpy as np
import joblib

In [2]:
import pandas as pd

In [3]:
def load_tif(file):
    ds = gdal.Open(file)
    arr = ds.GetRasterBand(1).ReadAsArray()
    return arr

# Give full path to pre-trained model in next cell

In [4]:
### Load pre-trained model
m = joblib.load("rf_test.joblib")

# Change path to point to your geoTiffs in next cell 

In [5]:
base_dir = "/att/nobackup/jli30/workspace/ndvi_MGBrown_notebooks/data/test_out/" 

regions = ["podor",  "delta", "west"] # AOIs

year = 2006
vihis = 5 # past 5 years NDVI
tint = 30 # past 30 days Precip

In [6]:
aoi = regions[0]
days = np.arange(1,365,16) # MOD13 freq 

In [7]:
# load landcover raster 
path = os.path.join(base_dir,
                   "LANDCOVER",
                   f"*{str(year)}*clip{aoi}.tif")
lc_file = glob.glob(path)[0]
if lc_file:
    lc = load_tif(lc_file)
else:
    raise RuntimeError("Can not find LandCover")

In [8]:
lc.shape

(12, 17)

In [15]:
for day in days[5:6]:
    
    yj = str(year)+str(day).zfill(3)
    c_date = datetime.datetime.strptime(yj, "%Y%j")
    print("Current Date: ", c_date)
    
    #--------------------------------------------------------------------------
    # Load NDVI (current + past 5 years)
    #--------------------------------------------------------------------------
    ndvi_date_p = [c_date.replace(year=c_date.year-i) for i in range(vihis+1)]   
    ndvi_list = []
    for d in ndvi_date_p:
        yyyy = str(d.year)
        ddd = str(day).zfill(3)
        path = os.path.join(base_dir,
                           "NDVI",
                           f"*{yyyy}{ddd}*.clip{aoi}.tif")
        f = glob.glob(path)[0]
        if f:
            ndvi = load_tif(f)
            ndvi_list.append(ndvi)
        else:
            date = d.strftime("Y%m%d")
            raise RuntimeError(f"Can not find NDVI for Date : {date}")
    
    #--------------------------------------------------------------------------
    # Load Precipitation (current + past 30 days)
    #--------------------------------------------------------------------------
    pr_date_p = [c_date-datetime.timedelta(days=i) for i in range(tint+1)]
    pr_list = []
    for d in pr_date_p:
        yyyy = str(d.year)
        tstamp = datetime.datetime.strftime(d, "%Y%m%d")
        path = os.path.join(base_dir,
                           "PRECIP",
                           f"*{tstamp}*.clip{aoi}.tif")
        f = glob.glob(path)[0]
        if f:
            pr = load_tif(f)
            pr_list.append(pr)
        else:
            date = d.strftime("Y%m%d")
            raise RuntimeError(f"Can not find Precip for Date : {date}")
    # accumulate precip
    p30 = np.sum(np.array(pr_list), axis=0)
    
    #--------------------------------------------------------------------------
    # Form ind-variable dataframe
    #--------------------------------------------------------------------------
    # create mask based on current ndvi & lc
    vi = ndvi_list[0]
    mask = np.logical_and(lc!=12, vi>0)
    uvi = np.where(mask, vi, np.nan)
    
    # apply mask for other variables
    colnames= ['upre', 'upvi', 'u2pvi', 'u3pvi', 'u4pvi', 'u5pvi']
    X = pd.DataFrame(columns = colnames)
    
    upre = np.where(mask, p30, np.nan)
    X['upre'] = upre.ravel()
    for i, key in enumerate(colnames[1:]):
        arr = np.where(mask, ndvi_list[i+1], np.nan)
        X[key] = arr.ravel()
    
    #--------------------------------------------------------------------------
    # Make prediction
    #--------------------------------------------------------------------------
    X = X.dropna()
    y_pred = m.predict(X)


Current Date:  2006-03-22 00:00:00


In [16]:
X

,upre,upvi,u2pvi,u3pvi,u4pvi,u5pvi
0,172.164688,1660.0,1855.0,1541.0,1892.0,1750.0
1,170.580078,1682.0,1855.0,1854.0,2680.0,1952.0
2,168.995483,1704.0,2342.0,1806.0,1901.0,1883.0
3,167.410889,-382.0,-988.0,2027.0,2954.0,-1867.0
4,165.826279,2397.0,2156.0,2041.0,-958.0,-161.0
...,...,...,...,...,...,...
186,161.864563,4291.0,2592.0,2258.0,3582.0,2096.0
197,162.733841,2139.0,2450.0,2867.0,2117.0,1896.0
198,162.733841,2934.0,3344.0,2949.0,3492.0,1908.0
199,162.733841,788.0,2773.0,1955.0,3757.0,3355.0


In [17]:
y_pred

array([1884.15, 2287.75, 2171.55,  111.5 ,  -80.15, 3456.4 , 3887.05,
       1822.3 , 1897.55, 1886.55, 2009.6 , 1982.95, 4685.9 , 2452.65,
        870.35,  573.75, 3403.15, 3355.55, 3060.25, 1939.25, 1968.55,
       1977.6 , 2047.8 , 1934.5 , 1821.85, 1770.95, 1816.3 , 2386.  ,
       1779.6 , 2647.7 , -101.45, -148.45, 2103.15, 2580.6 , 2584.2 ,
       1956.8 , 1926.45, 1968.8 , 2939.7 , 2746.6 , 2469.5 , 3576.  ,
       2309.2 , 1899.7 , 2669.35, 2962.4 , 3340.2 ,  369.5 , 4134.05,
       2083.7 , 3260.15, 1760.7 , 2408.2 , 2380.5 , 2686.6 , 2528.6 ,
       2189.4 , 3117.75, 4522.55, 1869.65, 2488.6 , 2335.5 , 4950.6 ,
       -221.05, 1901.95, 2165.75, 1808.4 , 2760.25, 2416.1 , 3593.95,
       2236.  , 2145.6 , 1981.25, 3674.25, 3154.2 , 1845.6 , 2400.  ,
       1788.5 , 3799.35, 2115.1 , 3517.35,  161.05, 2235.95, 3106.  ,
       2380.85, 4303.3 , 2780.85, 2494.6 , 2325.65, 3213.85, 3800.7 ,
       1826.8 , 3247.05, 2322.7 , 3498.65, 2646.7 , 1955.25, 2034.5 ,
        508.4 , 2824